In [1]:
import pandas as pd
import numpy as np
import tarfile
%reload_ext version_information
np.__version__, pd.__version__

('1.16.4', '0.24.2')

<h3>Einlesen der .tgz-Datei mithilfe von <I>tarfile</I></h3>

In [2]:
tar = tarfile.open("lfw-funneled.tgz")
#tar.extractall() #gefährlich, entpackt direkt alles

In [3]:
i = 0
for tarinfo in tar:
    print(tarinfo.name, "is", tarinfo.size, "bytes in size and is ", end="")
    if tarinfo.isreg():
        print("a regular file.")
    elif tarinfo.isdir():
        print("a directory.")
    else:
        print("something else.")
        
    i+=1
    if i == 20:
        break


lfw_funneled is 0 bytes in size and is a directory.
lfw_funneled/George_HW_Bush is 0 bytes in size and is a directory.
lfw_funneled/George_HW_Bush/George_HW_Bush_0001.jpg is 17691 bytes in size and is a regular file.
lfw_funneled/George_HW_Bush/George_HW_Bush_0002.jpg is 14895 bytes in size and is a regular file.
lfw_funneled/George_HW_Bush/George_HW_Bush_0003.jpg is 18220 bytes in size and is a regular file.
lfw_funneled/George_HW_Bush/George_HW_Bush_0004.jpg is 17493 bytes in size and is a regular file.
lfw_funneled/George_HW_Bush/George_HW_Bush_0005.jpg is 21517 bytes in size and is a regular file.
lfw_funneled/George_HW_Bush/George_HW_Bush_0006.jpg is 17631 bytes in size and is a regular file.
lfw_funneled/George_HW_Bush/George_HW_Bush_0007.jpg is 20535 bytes in size and is a regular file.
lfw_funneled/George_HW_Bush/George_HW_Bush_0008.jpg is 17979 bytes in size and is a regular file.
lfw_funneled/George_HW_Bush/George_HW_Bush_0009.jpg is 16926 bytes in size and is a regular file.

<h3>2a) Zwischenspeichern der Personen mit mindestens 70 Bildern</h3>

In [26]:
i = 0
popular = []
for tarinfo in tar:
    if tarinfo.isdir():
        person = tarinfo.name
        i = 0
    elif tarinfo.isreg() and tarinfo.name.endswith("jpg"):
        i += 1
    if i >= 70:
        if person not in popular:
            popular.append(person)
            print(person)
        

['lfw_funneled/Gerhard_Schroeder', 70]
['lfw_funneled/Gerhard_Schroeder', 70, 'lfw_funneled/George_W_Bush', 70]
['lfw_funneled/Gerhard_Schroeder', 70, 'lfw_funneled/George_W_Bush', 70, 'lfw_funneled/Donald_Rumsfeld', 70]
['lfw_funneled/Gerhard_Schroeder', 70, 'lfw_funneled/George_W_Bush', 70, 'lfw_funneled/Donald_Rumsfeld', 70, 'lfw_funneled/Tony_Blair', 70]
['lfw_funneled/Gerhard_Schroeder', 70, 'lfw_funneled/George_W_Bush', 70, 'lfw_funneled/Donald_Rumsfeld', 70, 'lfw_funneled/Tony_Blair', 70, 'lfw_funneled/Ariel_Sharon', 70]
['lfw_funneled/Gerhard_Schroeder', 70, 'lfw_funneled/George_W_Bush', 70, 'lfw_funneled/Donald_Rumsfeld', 70, 'lfw_funneled/Tony_Blair', 70, 'lfw_funneled/Ariel_Sharon', 70, 'lfw_funneled/Colin_Powell', 70]
['lfw_funneled/Gerhard_Schroeder', 70, 'lfw_funneled/George_W_Bush', 70, 'lfw_funneled/Donald_Rumsfeld', 70, 'lfw_funneled/Tony_Blair', 70, 'lfw_funneled/Ariel_Sharon', 70, 'lfw_funneled/Colin_Powell', 70, 'lfw_funneled/Hugo_Chavez', 70]


<h3>2b) Zwischenspeichern der Bilder der am häufigsten abgebildeten Personen</h3>

Je ein Bild wird für die Klassifikation beiseite gelegt.

Zuerst Zwischenspeichern der Namen der Bilddateien, nach Personen getrennt, in einer Liste von Listen

In [27]:
popular_b = False
count = 0
all_cnt = []
pictures = []
pct = []
for tarinfo in tar:
    if tarinfo.isdir() and tarinfo.name in popular:
        popular_b = True
    elif tarinfo.isdir() and tarinfo.name not in popular:
        if popular_b:
            all_cnt.append(count)
            count = 0
            pictures.append(pct)
            pct = []

        popular_b = False
        
    if popular_b == True and tarinfo.name.endswith("jpg"):
        count += 1
        pct.append(tarinfo.name)
        #print(tarinfo.name)

In [42]:
all_cnt

[109, 530, 121, 144, 77, 236, 71]

In [43]:
for plist in pictures:
    print(len(plist))

109
530
121
144
77
236
71


In [5]:
tar.close()

In [7]:
%version_information numpy, pandas

Software versions
Python 3.7.3 64bit [MSC v.1915 64 bit (AMD64)]
IPython 7.6.1
OS Windows 10 10.0.18362 SP0
numpy 1.16.4
pandas 0.24.2
Mon Nov 04 08:33:51 2019 Mitteleuropäische Zeit